In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import pickle

import sys
sys.path.append('../../')
from scripts.viz_tools import animate_patch
from scripts.dl_utils import download_patch, rect_from_point

In [ ]:
# This function takes a while to run as it is extracting data from GEE

rect_width = 0.002
coords = [-104.40007446321363, 38.94994987109655]
start_date = '2016-01-01'
end_date = '2021-09-01'
patches, raster_info = download_patch(rect_from_point(coords, rect_width), start_date, end_date)
print(len(patches))

In [ ]:
name = "CO_field"
cloud_threshold = .2
file_name = f"../../figures/videos/{name} {coords[1]:.2f}, {coords[0]:.2f} - thresh {cloud_threshold} - {start_date}-{end_date}.mp4"
animate_patch(patches, file_name, cloud_threshold=cloud_threshold, interval=50, size=8)

## Export Frames as Images

In [ ]:
from scripts import viz_tools
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image

images = []
stretch = True
folder_name = file_name = f"../../figures/frames/{name} {coords[1]:.2f}, {coords[0]:.2f} - thresh {cloud_threshold} - {start_date}-{end_date}"
if not os.path.exists(folder_name):
    os.mkdirs(folder_name)
for index, img in enumerate(patches):
    if np.sum(img.mask) / np.size(img.mask) < cloud_threshold:
        rgb = viz_tools.normalize(img[:,:,3:0:-1])
        if stretch:
            rgb = viz_tools.stretch_histogram(rgb, 0.1, 1.0, gamma=1.2)
        im = Image.fromarray(np.uint8((np.clip(rgb, 0, 1) * 255)))
        im.save(f'{folder_name}/frame_{index}.png')